In [2]:
import os
import sys


sys.path.append(os.path.abspath("../..")) #make the folder "automl" part of this

In [3]:
from automl.utils.files_utils import open_or_create_folder


base_experiment_path = "data\\plays"
#multiple_experiment_folder_name = "multiple_plays"
multiple_experiment_folder_name = "sb3_models_comparison"
experiment_path = open_or_create_folder(f"{base_experiment_path}\\{multiple_experiment_folder_name}")
device = "cpu"

In [4]:
NUMBER_OF_ENVIRONMENTS = 3

## Load Models / Policies

In [5]:
import os

models_folder = 'data\\models'
model_paths = [
        os.path.join(models_folder, f)
        for f in os.listdir(models_folder)
        if os.path.isfile(os.path.join(models_folder, f))
    ]

#model_paths = model_paths[:4]

print(f"Model paths:\n{model_paths}")

Model paths:
['data\\models\\sb3_mountaincar_dqn.pkl', 'data\\models\\sb3_mountaincar_dqn_perturbed_10_10000_gaussian_0_0.1_0.9.pkl']


In [6]:
model_names = [os.path.splitext(os.path.basename(path))[0] for path in model_paths]

print("Model names:\n", model_names)

Model names:
 ['sb3_mountaincar_dqn', 'sb3_mountaincar_dqn_perturbed_10_10000_gaussian_0_0.1_0.9']


In [7]:
experiments_for_models_paths = []

In [8]:
def load_model(index):
    return model_paths[index]

In [9]:
from automl.rl.policy.qpolicy import QPolicy

def load_policy(index):
    return (QPolicy, {"model" : load_model(index)})

## Setup Environment

In [10]:
from automl.rl.environment.gymnasium_env import GymnasiumEnvironmentWrapper

base_env_definition = (GymnasiumEnvironmentWrapper, 
                       {"environment" : "MountainCar-v0", 
                        #"render_mode" : "human", 
                        "device" : device})


In [11]:
env_list = [base_env_definition for _ in range(NUMBER_OF_ENVIRONMENTS)]

In [12]:
from automl.rl.environment.environment_sampler import EnvironmentCycler


env = EnvironmentCycler()

env.pass_input({
    "environments" : env_list,
    "generate_name" : True
})

## Setup Single Agent

In [13]:
from automl.rl.agent.agent_components import AgentSchema

def single_agent_fun(index):

    policy_input = {}


    agent_input = {
        "policy": load_policy(index),

    }

    agent = (AgentSchema, 
             agent_input)
    
    return agent

## Setup all Agents

In [14]:
all_agents_input = {
    "device" : device,
}

In [15]:
def agents_fun(index):
    return single_agent_fun(index)

## Setup player

In [16]:
from automl.rl.rl_player.rl_player import RLPlayer

def rl_player_fun(index):

    rl_player_input = {
        "base_directory" : experiment_path,
        "artifact_relative_directory" : "play",
        "create_new_directory" : True,
        "agents_input" : all_agents_input,
        "num_episodes" : 5,
        "store_env_at_end" : True    
        }

    rl_player_class = RLPlayer

    return (rl_player_class, rl_player_input)

In [17]:
from automl.rl.evaluators.rl_evaluator_player import EvaluatorWithPlayer

def eval_with_player(index):

    return EvaluatorWithPlayer({
        
                "rl_player_definition" : rl_player_fun(index),
                "number_of_episodes" : 3, # number of episodes per environment
                "number_of_evaluations" : NUMBER_OF_ENVIRONMENTS # number of sampled environments essentially
    
    })


## Play

In [18]:
from automl.utils.files_utils import open_or_create_folder
from automl.utils.json_component_utils import save_configuration

print(f"Using experiment path {experiment_path} from base experiment path {experiment_path}")

for index in range(len(model_paths)):

    experiment_path_index = open_or_create_folder(f"{experiment_path}\\{model_names[index]}", create_new=True)

    experiments_for_models_paths.append(experiment_path_index)

    evaluator_with_player = eval_with_player(index)

    agents = agents_fun(index)

    evaluator_with_player.evaluate((agents, device, experiment_path_index, env))

    save_configuration(evaluator_with_player, experiment_path_index, "config.json", save_exposed_values=True, ignore_defaults=False)

Using experiment path data\plays\sb3_models_comparison from base experiment path data\plays\sb3_models_comparison


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception:

saving configuration


c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\Universidade\ricardo-goncalo-thesis-project\project\automl\loggers\result_logger.py:78

saving configuration


...

c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\Universidade\ricardo-goncalo-thesis-project\project\automl\loggers\result_logger.py:78

saving configuration


c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\Universidade\ricardo-goncalo-thesis-project\project\automl\loggers\result_logger.py:78

saving configuration
saving configuration


c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\anaconda3\envs\AIPython\Lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code expected at least 16 arguments, got 15
  warnings.warn(
c:\Users\rgoncalo\Universidade\ricardo-goncalo-thesis-project\project\automl\loggers\result_logger.py:78

## Store results

In [19]:

from automl.loggers.result_logger import aggregate_results_logger


joint_results = aggregate_results_logger(experiments_for_models_paths, experiment_path)


In [20]:
joint_evaluations = aggregate_results_logger(experiments_for_models_paths, experiment_path, ("Model", model_names), results_filename="evaluation.csv")